In [51]:
import pandas as pd
import numpy as np
import tqdm
import scipy.stats as sts

In [71]:
panel_data = pd.read_csv(
    "./large_data/intermediate_data/panel_data.csv", index_col=0
).drop(columns=["transaction_date.1","purchase_sum.1","treatment_flg.1","female.1","male.1","city_type.1"])

In [73]:
panel_data

,client_id,transaction_date,city_type,treatment_flg,age,female,male,purchase_sum,avg_product_quantity,total_products,ch_avg_purchase,unique_segments,unique_brands,alcohol_percentage,own_trademark_percentage
0,000012768d,2018-12-01,1.0,0.0,45,0.0,0.0,1007.000000,1.105263,21.0,1007.0,13,13,0.000000,10.526316
1,000012768d,2018-12-16,1.0,0.0,45,0.0,0.0,574.000000,1.272727,14.0,574.0,9,9,0.000000,9.090909
2,000012768d,2019-03-08,1.0,0.0,45,0.0,0.0,803.000000,0.812500,13.0,803.0,13,13,0.000000,6.250000
3,000012768d,2019-03-14,1.0,0.0,45,0.0,0.0,419.000000,1.000000,6.0,419.0,6,6,0.000000,0.000000
4,000036f903,2018-11-28,0.0,0.0,72,1.0,0.0,241.000000,0.800000,4.0,241.0,5,5,0.000000,40.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7145657,fffff6ce77,2019-02-27,1.0,1.0,42,0.0,0.0,860.331587,1.000000,5.0,424.0,5,5,15.789474,40.000000
7145658,fffff6ce77,2019-02-28,1.0,1.0,42,0.0,0.0,1109.782880,1.111111,10.0,446.0,8,7,11.111111,50.000000
7145659,fffff6ce77,2019-03-05,1.0,1.0,42,0.0,0.0,385.217761,1.000000,1.0,247.0,1,1,15.789474,0.000000
7145660,fffff6ce77,2019-03-06,1.0,1.0,42,0.0,0.0,1117.504754,1.000000,3.0,177.0,2,2,15.789474,50.000000


In [75]:
poor = panel_data.query('city_type == 0')
rich = panel_data.query('city_type == 1')

**собираем матрицу для модели пространства состояний**

In [77]:
rich_lss_raw = rich.query('treatment_flg == 0')

rich_lss_raw['lag1_purchase_sum'] = rich_lss_raw.groupby('client_id').purchase_sum.shift(1)
rich_lss_raw.dropna(inplace=True)

/var/folders/t0/4q3bn69d7nzbpxs2hdjgt76m0000gn/T/ipykernel_17797/1426819003.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rich_lss_raw['lag1_purchase_sum'] = rich_lss_raw.groupby('client_id').purchase_sum.shift(1)
/var/folders/t0/4q3bn69d7nzbpxs2hdjgt76m0000gn/T/ipykernel_17797/1426819003.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rich_lss_raw.dropna(inplace=True)


In [82]:
rich_lss = (
    rich_lss_raw.drop(columns=["client_id", "city_type", "treatment_flg"])
    .groupby("transaction_date")
    .mean()
)

In [84]:
rich_lss.head()

,age,female,male,purchase_sum,avg_product_quantity,total_products,ch_avg_purchase,unique_segments,unique_brands,alcohol_percentage,own_trademark_percentage,lag1_purchase_sum
transaction_date,,,,,,,,,,,,
2018-11-22,31.500000,0.000000,0.500000,599.810000,1.148352,11.500000,599.810000,8.500000,9.500000,0.000000,14.835165,579.965000
2018-11-23,44.143718,0.413074,0.183588,636.285318,1.254620,9.741307,559.430763,6.274455,6.175707,2.237526,15.236285,615.879356
2018-11-24,44.046174,0.378255,0.191141,657.156177,1.272270,10.048859,571.600806,6.411544,6.313289,2.250577,15.686392,638.582966
2018-11-25,44.126712,0.379522,0.194972,633.109174,1.248656,9.973431,562.702023,6.464132,6.328224,1.815559,14.660032,675.194502
2018-11-26,45.413244,0.394132,0.180889,585.001355,1.255363,9.683990,527.059112,6.286672,6.092875,1.291168,14.444494,653.400573


**Linear State Space model**

In [87]:
def estimate_lss_matrices(x, y):
    """
    Оценка матриц модели Linear State Space: A, C, Q, R.

    Параметры:
    - x: ndarray (n_states, T) - матрица скрытых состояний x_t
    - y: ndarray (n_obs, T) - матрица наблюдений y_t

    Возвращает:
    - A, C: матрицы перехода и наблюдений
    - Q, R: ковариации шумов состояний и наблюдений
    """
    T = x.shape[1]  # Количество временных шагов
    n_states = x.shape[0]
    n_obs = y.shape[0]

    # Оценка A: матрица перехода состояний
    x_t = x[:, :-1]      # x_t (n_states, T-1)
    x_next = x[:, 1:]    # x_{t+1} (n_states, T-1)
    A = x_next @ x_t.T @ np.linalg.inv(x_t @ x_t.T)

    # Оценка C: матрица наблюдений
    C = y @ x.T @ np.linalg.inv(x @ x.T)

    # Оценка Q: ковариация шума состояний w_t
    w = x_next - A @ x_t  # Остатки для состояний
    Q = (w @ w.T) / (T - 1)

    # Оценка R: ковариация шума наблюдений v_t
    v = y - C @ x          # Остатки для наблюдений
    R = (v @ v.T) / T

    return A, C, Q, R

In [89]:
Y_tr = rich_lss['purchase_sum'].values.reshape(1, -1)

X_tr = rich_lss[['age', 'female', 'male', 'avg_product_quantity',
       'total_products', 'ch_avg_purchase', 'unique_segments', 'unique_brands',
       'alcohol_percentage', 'own_trademark_percentage',
       'lag1_purchase_sum']].values.T.reshape(len(rich_lss.columns) - 1, -1)

**оценка и корректировка параметров**

In [91]:
A_est_tr, C_est_tr, Q_est_tr, R_est_tr = estimate_lss_matrices(X_tr, Y_tr)
Q_est_tr += np.eye(11)*0.15 # так надо 

**Kalman's filter**

In [94]:
def kalman_filter(y, A, C, Q, R, X, agg_mtx):
    """
    Фильтр Калмана для восстановления скрытых состояний x_t на основе наблюдений y_t.

    Параметры:
    - y: ndarray (n_obs, T) - матрица наблюдений
    - A: ndarray (n_states, n_states) - матрица перехода состояний
    - C: ndarray (n_obs, n_states) - матрица наблюдений
    - Q: ndarray (n_states, n_states) - ковариация шума состояний
    - R: ndarray (n_obs, n_obs) - ковариация шума наблюдений
    - x0: ndarray (n_states,) - начальное состояние
    - P0: ndarray (n_states, n_states) - начальная ковариация ошибки

    Возвращает:
    - x_est: ndarray (n_states, T) - оценки скрытых состояний
    - P_est: list - ковариации ошибок для каждого времени
    """

    x0 = pd.DataFrame(X).mean(axis = 1).values 
    P0 = pd.DataFrame(X).T.cov().values.reshape(len(agg_mtx.columns) - 1, len(agg_mtx.columns) - 1)
    
    T = y.shape[1]  # Количество временных шагов
    n_states = A.shape[0]

    # Инициализация
    x_est = np.zeros((n_states, T))
    P_est = []

    x_pred = x0
    P_pred = P0

    for t in range(T):
        # Шаг обновления (Update)
        K = P_pred @ C.T @ np.linalg.inv(C @ P_pred @ C.T + R)  # Коэффициент Калмана
        x_upd = x_pred + K @ (y[:, t] - C @ x_pred)             # Обновление состояния
        P_upd = (np.eye(n_states) - K @ C) @ P_pred             # Обновление ковариации ошибки

        # Сохранение оценок
        x_est[:, t] = x_upd
        P_est.append(P_upd)

        # Шаг предсказания (Predict)
        if t < T - 1:
            x_pred = A @ x_upd
            P_pred = A @ P_upd @ A.T + Q

    return x_est, P_est

In [96]:
observ_data = poor.groupby(['client_id', 'transaction_date'], as_index=False).purchase_sum.sum()
observ_data.head()

,client_id,transaction_date,purchase_sum
0,000036f903,2018-11-28,241.0
1,000036f903,2018-12-03,458.0
2,000036f903,2018-12-06,328.0
3,000036f903,2018-12-10,127.0
4,000036f903,2018-12-12,280.0


In [97]:
def restore_poor_treat_features(poor_treat, kalman_filter, A, C, Q, R, X, agg_mtx):
    """
    Функция для восстановления признаков клиентов с использованием фильтра Калмана.

    Parameters:
    - poor_treat (pd.DataFrame): Исходный DataFrame с клиентами и их покупками.
    - kalman_filter (function): Функция фильтра Калмана.
    - A_est_tr, C_est_tr, Q_est_tr, R_est_tr: Параметры фильтра Калмана.
    - X_tr: Начальные значения для фильтра Калмана.
    - lss_treat (pd.DataFrame): Матрица агрегированных данных.

    Returns:
    - restored_df_poor_treat (pd.DataFrame): DataFrame с восстановленными признаками.
    """
    restored_df = pd.DataFrame([])

    # Перебор всех уникальных клиентов
    for client in tqdm.tqdm(poor_treat.client_id.unique()[:30_000]):
        client_purchase = poor_treat[poor_treat.client_id == client]
        Y = client_purchase.purchase_sum.values.reshape(1, client_purchase.shape[0])

        # Применение фильтра Калмана
        x_est, P_est = kalman_filter(Y, A, C, Q, R, X, agg_mtx)

        # Усреднение признаков клиента
        mean_client_feat = pd.DataFrame(x_est.mean(axis=1)).T
        mean_client_feat.columns = agg_mtx.drop(columns=['purchase_sum']).columns
        mean_client_feat['client_id'] = client

        # Обновление итогового DataFrame
        restored_df = pd.concat([restored_df, mean_client_feat])

    # Сброс индексовÍ›
    restored_df.reset_index(drop=True, inplace=True)
    
    return restored_df

In [100]:
restored_df_poor = restore_poor_treat_features(
    poor_treat = observ_data,
    kalman_filter = kalman_filter,
    A = A_est_tr,
    C = C_est_tr,
    Q = Q_est_tr,
    R = R_est_tr,
    X = X_tr,
    agg_mtx = rich_lss,
)

restored_df_poor.to_csv('./large_data/output_data/restored_poor_people.csv', index=None)

100%|█████████████████████████████████████| 30000/30000 [47:39<00:00, 10.49it/s]
